In [153]:
import opennre
import itertools
from utils import wikifier, coref_resolution, strip_punctuation
import spacy
import neuralcoref
import nltk
import transformers
from config import APPO, STOPWORDS, TOKENIZER
import re

relation_model = opennre.get_model('wiki80_bert_softmax')
#wiki80_bertentity_softmax => poor performance

2021-03-19 14:57:03,797 - root - INFO - Loading BERT pre-trained checkpoint.


In [154]:
#nlp = spacy.load("en_core_web_sm")
#doc = nlp(text)
#doc._.has_coref
#doc._.coref_clusters
#neuralcoref.add_to_pipe(nlp)

In [155]:
text = "Elon Musk is a business magnate, industrial designer, and engineer. Elon Musk is the founder, CEO, CTO, and chief designer of SpaceX. Elon Musk is also early investor, CEO, and product architect of Tesla , Inc. Elon Musk is also the founder of The Boring Company and the co-founder of Neuralink. A centibillionaire, Elon Musk became the richest person in the world in January 2021, with an estimated net worth of $185 billion at the time, surpassing Jeff Bezos. Elon Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. Elon Musk briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen ' s University. Elon Musk transferred to the University of Pennsylvania two years later, where Elon Musk received dual bachelor's degrees in economics and physics. Elon Musk moved to California in 1995 to attend Stanford University, but decided instead to pursue a business career. Elon Musk went on co-founding a web software company Zip2 with Elon Musk brother Kimbal Musk."

text = coref_resolution(text)
text

"Elon Musk is a business magnate, industrial designer, and engineer. Elon Musk is the founder, CEO, CTO, and chief designer of SpaceX. Elon Musk is also early investor, CEO, and product architect of Tesla , Inc. Elon Musk is also the founder of The Boring Company and the co-founder of Neuralink. A centibillionaire, Elon Musk became the richest person in the world in January 2021, with an estimated net worth of $185 billion at the time, surpassing Jeff Bezos. Elon Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. Elon Musk briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen ' s University. Elon Musk transferred to the University of Pennsylvania two years later, where Elon Musk received dual bachelor's degrees in economics and physics. Elon Musk moved to California in 1995 to attend Stanford University, but decided instead to pursue a business career. Elon Musk went on co-founding a web software compa

In [156]:
ner = transformers.pipeline('ner', grouped_entities=True)
entities = ner(text)

In [157]:
subjects = dict()
for entity in entities:
    if entity["word"] not in subjects:
        subjects[entity["word"]] = {
            "type": set([entity["entity_group"]])
        }
    else:
        subjects[entity["word"]]["type"].add(entity["entity_group"])

for key in subjects:
    subjects[strip_punctuation(key)] = subjects.pop(key)
    
sentences = [strip_punctuation(s) for s in nltk.sent_tokenize(text)]

for i, sentence in enumerate(sentences):
    for key in subjects.keys():

        if key not in sentence:
            continue
        
        key_len = len(key)
        pos = [(m.start(), m.start() + key_len) for m in re.finditer(key, sentence)]
        if "pos" not in subjects[key]:
            subjects[key]["pos"] = dict()
        subjects[key]["pos"][i] = pos

In [161]:
#triples = set()
triples = dict()
relation_threshold = 0.75
relation_model = opennre.get_model('wiki80_bert_softmax')

for i, sentence in enumerate(sentences):
    for subj in subjects.keys():
        for obj in subjects.keys():

            if subj == obj:
                continue
            
            if (i not in subjects[subj]["pos"]) or (i not in subjects[obj]["pos"]):
                continue

            for pos_subject in subjects[subj]["pos"][i]:
                for pos_object in subjects[obj]["pos"][i]:
                    data = relation_model.infer({'text': sentence, 'h': {'pos': pos_subject}, 't': {'pos': pos_object}})

                    if data[1] > relation_threshold:
                        triple = (subj, data[0], obj)
                        #triples.add((subj, data[0], obj))
                        if (not triple in triples) or (triples[triple] < data[1]):
                            triples[triple] = data[1]

triples = [(k, v) for k, v in sorted(triples.items(), key=lambda x: x[1], reverse=True)]

2021-03-19 15:05:36,759 - root - INFO - Loading BERT pre-trained checkpoint.
